# Customer Churn Prediction - Model Development

This notebook implements and evaluates various machine learning models for predicting customer churn.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
from imblearn.over_sampling import SMOTE

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Create directories for saving outputs
os.makedirs('../models', exist_ok=True)
os.makedirs('../docs/plots', exist_ok=True)

## 1. Load the Engineered Data

In [ ]:
# Load the engineered data
df_engineered = pd.read_csv('../data/processed/churn_engineered.csv')

# Display basic information
print(f"Dataset shape: {df_engineered.shape}")
print(f"\nColumns: {df_engineered.columns.tolist()}")
print(f"\nSample data:")
df_engineered.head()

## 2. Load Feature Sets

In [ ]:
# Load feature sets
with open('../models/feature_sets.json', 'r') as f:
    feature_sets = json.load(f)

# Print feature set sizes
for name, features in feature_sets.items():
    print(f"{name}: {len(features)} features")

## 3. Prepare Data for Modeling

In [ ]:
# Function to prepare data for a specific feature set
def prepare_data(df, feature_set_name, test_size=0.2, random_state=42, apply_smote=False):
    # Get features for the specified feature set
    features = feature_sets[feature_set_name]
    
    # Prepare features and target
    X = df[features]
    y = df['Exited']
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    # Apply SMOTE to handle class imbalance if requested
    if apply_smote:
        smote = SMOTE(random_state=random_state)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print(f"After SMOTE - Class distribution: {pd.Series(y_train).value_counts()}")
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Check class distribution
print("Class distribution:")
print(df_engineered['Exited'].value_counts())
print(f"Churn rate: {df_engineered['Exited'].mean() * 100:.2f}%")

# Prepare data for modeling using the selected feature set
feature_set_name = 'selected_top'  # We'll use the selected top features
X_train, X_test, y_train, y_test = prepare_data(df_engineered, feature_set_name, apply_smote=True)

## 4. Model Development

### 4.1 Logistic Regression (Baseline)

In [ ]:
# Train Logistic Regression model
lr_model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
lr_model.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test)
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Logistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_lr):.4f}")

# Confusion matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Logistic Regression')
plt.tight_layout()
plt.savefig('../docs/plots/lr_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

# Save the model
with open('../models/logistic_regression.pkl', 'wb') as f:
    pickle.dump(lr_model, f)
print("Logistic Regression model saved to ../models/logistic_regression.pkl")

### 4.2 Random Forest

In [ ]:
# Train Random Forest model
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Random Forest Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_rf):.4f}")

# Confusion matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Random Forest')
plt.tight_layout()
plt.savefig('../docs/plots/rf_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
plt.title('Random Forest Feature Importance', fontsize=15)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('../docs/plots/rf_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Save the model
with open('../models/random_forest.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
print("Random Forest model saved to ../models/random_forest.pkl")

### 4.3 XGBoost

In [ ]:
# Train XGBoost model
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=4)  # scale_pos_weight helps with class imbalance
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("XGBoost Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_xgb):.4f}")

# Confusion matrix
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - XGBoost')
plt.tight_layout()
plt.savefig('../docs/plots/xgb_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
plt.title('XGBoost Feature Importance', fontsize=15)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('../docs/plots/xgb_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Save the model
with open('../models/xgboost.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
print("XGBoost model saved to ../models/xgboost.pkl")

### 4.4 Support Vector Machine

In [ ]:
# Train SVM model
svm_model = SVC(random_state=42, probability=True, class_weight='balanced')
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)
y_prob_svm = svm_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("SVM Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_svm):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_svm):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_svm):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_svm):.4f}")

# Confusion matrix
cm_svm = confusion_matrix(y_test, y_pred_svm)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - SVM')
plt.tight_layout()
plt.savefig('../docs/plots/svm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

# Save the model
with open('../models/svm.pkl', 'wb') as f:
    pickle.dump(svm_model, f)
print("SVM model saved to ../models/svm.pkl")

### 4.5 Neural Network

In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(random_state=42, max_iter=1000, hidden_layer_sizes=(100, 50), alpha=0.01)
nn_model.fit(X_train, y_train)

# Make predictions
y_pred_nn = nn_model.predict(X_test)
y_prob_nn = nn_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Neural Network Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nn):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_nn):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_nn):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_nn):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_nn):.4f}")

# Confusion matrix
cm_nn = confusion_matrix(y_test, y_pred_nn)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_nn, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Neural Network')
plt.tight_layout()
plt.savefig('../docs/plots/nn_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_nn))

# Save the model
with open('../models/neural_network.pkl', 'wb') as f:
    pickle.dump(nn_model, f)
print("Neural Network model saved to ../models/neural_network.pkl")

## 5. Model Comparison

In [ ]:
# Collect model results
models = {
    'Logistic Regression': {
        'y_pred': y_pred_lr,
        'y_prob': y_prob_lr,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred_lr),
            'precision': precision_score(y_test, y_pred_lr),
            'recall': recall_score(y_test, y_pred_lr),
            'f1': f1_score(y_test, y_pred_lr),
            'roc_auc': roc_auc_score(y_test, y_prob_lr)
        }
    },
    'Random Forest': {
        'y_pred': y_pred_rf,
        'y_prob': y_prob_rf,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred_rf),
            'precision': precision_score(y_test, y_pred_rf),
            'recall': recall_score(y_test, y_pred_rf),
            'f1': f1_score(y_test, y_pred_rf),
            'roc_auc': roc_auc_score(y_test, y_prob_rf)
        }
    },
    'XGBoost': {
        'y_pred': y_pred_xgb,
        'y_prob': y_prob_xgb,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred_xgb),
            'precision': precision_score(y_test, y_pred_xgb),
            'recall': recall_score(y_test, y_pred_xgb),
            'f1': f1_score(y_test, y_pred_xgb),
            'roc_auc': roc_auc_score(y_test, y_prob_xgb)
        }
    },
    'SVM': {
        'y_pred': y_pred_svm,
        'y_prob': y_prob_svm,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred_svm),
            'precision': precision_score(y_test, y_pred_svm),
            'recall': recall_score(y_test, y_pred_svm),
            'f1': f1_score(y_test, y_pred_svm),
            'roc_auc': roc_auc_score(y_test, y_prob_svm)
        }
    },
    'Neural Network': {
        'y_pred': y_pred_nn,
        'y_prob': y_prob_nn,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred_nn),
            'precision': precision_score(y_test, y_pred_nn),
            'recall': recall_score(y_test, y_pred_nn),
            'f1': f1_score(y_test, y_pred_nn),
            'roc_auc': roc_auc_score(y_test, y_prob_nn)
        }
    }
}

# Create a DataFrame for model comparison
metrics_df = pd.DataFrame({
    model_name: {metric: value for metric, value in model_info['metrics'].items()}
    for model_name, model_info in models.items()
}).T

# Format metrics for better readability
metrics_df = metrics_df.round(4)

# Display model comparison
print("Model Comparison:")
metrics_df

In [ ]:
# Visualize model comparison
plt.figure(figsize=(15, 10))

# Plot metrics comparison
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

for i, metric in enumerate(metrics):
    plt.subplot(2, 3, i+1)
    sns.barplot(x=metrics_df.index, y=metrics_df[metric], palette=colors)
    plt.title(f'{metric.upper()} Comparison', fontsize=14)
    plt.ylabel(metric.upper(), fontsize=12)
    plt.xticks(rotation=45)
    plt.ylim(0, 1)

plt.tight_layout()
plt.savefig('../docs/plots/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for model_name, model_info in models.items():
    fpr, tpr, _ = roc_curve(y_test, model_info['y_prob'])
    roc_auc = model_info['metrics']['roc_auc']
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=15)
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.savefig('../docs/plots/roc_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Select Best Model

In [ ]:
# Identify the best model based on ROC-AUC
best_model_name = metrics_df['roc_auc'].idxmax()
best_model_auc = metrics_df.loc[best_model_name, 'roc_auc']
best_model_info = models[best_model_name]

print(f"Best model: {best_model_name} with ROC-AUC: {best_model_auc:.4f}")

# Save best model information
best_model_info_dict = {
    'model_name': best_model_name,
    'metrics': best_model_info['metrics'],
    'feature_set': feature_set_name
}

with open('../models/best_model_info.json', 'w') as f:
    json.dump(best_model_info_dict, f)

print(f"Best model information saved to ../models/best_model_info.json")

# Visualize confusion matrix for best model
cm_best = confusion_matrix(y_test, best_model_info['y_pred'])
plt.figure(figsize=(8, 6))
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix - {best_model_name} (Best Model)')
plt.tight_layout()
plt.savefig('../docs/plots/best_model_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Cost-Benefit Analysis

In [ ]:
# Define cost assumptions
cost_losing_customer = 1000  # Cost of losing a customer ($)
cost_retention_program = 100  # Cost of retention program per customer ($)

# Function to calculate cost-benefit
def calculate_cost_benefit(y_true, y_pred, y_prob=None, threshold=0.5):
    if y_prob is not None:
        y_pred = (y_prob >= threshold).astype(int)
    
    # Calculate confusion matrix values
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Calculate costs
    cost_false_negatives = fn * cost_losing_customer  # Missed churn predictions
    cost_false_positives = fp * cost_retention_program  # Unnecessary retention actions
    cost_true_positives = tp * cost_retention_program  # Necessary retention actions
    
    # Calculate benefits
    benefit_true_positives = tp * cost_losing_customer  # Saved customers
    
    # Calculate net benefit
    total_cost = cost_false_negatives + cost_false_positives + cost_true_positives
    total_benefit = benefit_true_positives
    net_benefit = total_benefit - total_cost
    
    return {
        'total_cost': total_cost,
        'total_benefit': total_benefit,
        'net_benefit': net_benefit,
        'roi': (total_benefit / total_cost if total_cost > 0 else 0) - 1,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
    }

# Calculate cost-benefit for each model
cost_benefit_results = {}
for model_name, model_info in models.items():
    cost_benefit_results[model_name] = calculate_cost_benefit(y_test, model_info['y_pred'])

# Create DataFrame for cost-benefit comparison
cost_benefit_df = pd.DataFrame({
    model_name: {
        'Total Cost ($)': results['total_cost'],
        'Total Benefit ($)': results['total_benefit'],
        'Net Benefit ($)': results['net_benefit'],
        'ROI (%)': results['roi'] * 100
    }
    for model_name, results in cost_benefit_results.items()
}).T

# Format for better readability
cost_benefit_df['Total Cost ($)'] = cost_benefit_df['Total Cost ($)'].map('${:,.0f}'.format)
cost_benefit_df['Total Benefit ($)'] = cost_benefit_df['Total Benefit ($)'].map('${:,.0f}'.format)
cost_benefit_df['Net Benefit ($)'] = cost_benefit_df['Net Benefit ($)'].map('${:,.0f}'.format)
cost_benefit_df['ROI (%)'] = cost_benefit_df['ROI (%)'].map('{:.2f}%'.format)

# Display cost-benefit comparison
print("Cost-Benefit Analysis:")
cost_benefit_df

In [ ]:
# Visualize cost-benefit comparison
plt.figure(figsize=(12, 8))

# Extract numeric values for plotting
net_benefits = [results['net_benefit'] for results in cost_benefit_results.values()]
model_names = list(cost_benefit_results.keys())

# Create bar chart
sns.barplot(x=model_names, y=net_benefits)
plt.title('Net Benefit Comparison', fontsize=15)
plt.ylabel('Net Benefit ($)', fontsize=12)
plt.xlabel('Model', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(net_benefits):
    plt.text(i, v + 1000, f'${v:,.0f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('../docs/plots/cost_benefit_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Threshold Analysis

In [ ]:
# Analyze different probability thresholds for the best model
thresholds = np.arange(0.1, 1.0, 0.05)
threshold_results = []

for threshold in thresholds:
    # Get predictions using the threshold
    y_pred_threshold = (best_model_info['y_prob'] >= threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_threshold)
    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)
    f1 = f1_score(y_test, y_pred_threshold)
    
    # Calculate cost-benefit
    cost_benefit = calculate_cost_benefit(y_test, None, best_model_info['y_prob'], threshold)
    
    threshold_results.append({
        'threshold': threshold,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'net_benefit': cost_benefit['net_benefit']
    })

# Create DataFrame for threshold analysis
threshold_df = pd.DataFrame(threshold_results)

# Display threshold analysis
print("Threshold Analysis:")
threshold_df.round(4)

In [ ]:
# Visualize threshold analysis
plt.figure(figsize=(15, 10))

# Plot metrics vs threshold
plt.subplot(2, 1, 1)
plt.plot(threshold_df['threshold'], threshold_df['accuracy'], 'b-', label='Accuracy')
plt.plot(threshold_df['threshold'], threshold_df['precision'], 'g-', label='Precision')
plt.plot(threshold_df['threshold'], threshold_df['recall'], 'r-', label='Recall')
plt.plot(threshold_df['threshold'], threshold_df['f1'], 'y-', label='F1 Score')
plt.xlabel('Threshold', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Metrics vs Threshold', fontsize=15)
plt.grid(True, alpha=0.3)
plt.legend()

# Plot net benefit vs threshold
plt.subplot(2, 1, 2)
plt.plot(threshold_df['threshold'], threshold_df['net_benefit'], 'b-')
plt.xlabel('Threshold', fontsize=12)
plt.ylabel('Net Benefit ($)', fontsize=12)
plt.title('Net Benefit vs Threshold', fontsize=15)
plt.grid(True, alpha=0.3)

# Find optimal threshold based on net benefit
optimal_threshold_idx = threshold_df['net_benefit'].idxmax()
optimal_threshold = threshold_df.loc[optimal_threshold_idx, 'threshold']
optimal_net_benefit = threshold_df.loc[optimal_threshold_idx, 'net_benefit']

plt.axvline(x=optimal_threshold, color='r', linestyle='--')
plt.text(optimal_threshold + 0.02, optimal_net_benefit - 5000, 
         f'Optimal Threshold: {optimal_threshold:.2f}\nNet Benefit: ${optimal_net_benefit:,.0f}',
         fontsize=10)

plt.tight_layout()
plt.savefig('../docs/plots/threshold_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Optimal threshold: {optimal_threshold:.2f} with net benefit: ${optimal_net_benefit:,.0f}")

## 9. Save Results

In [ ]:
# Save model comparison results
metrics_df.to_csv('../models/model_comparison.csv')
print("Model comparison results saved to ../models/model_comparison.csv")

# Save cost-benefit analysis results
pd.DataFrame(cost_benefit_results).to_json('../models/cost_benefit_results.json')
print("Cost-benefit analysis results saved to ../models/cost_benefit_results.json")

# Save threshold analysis results
threshold_df.to_csv('../models/threshold_analysis.csv')
print("Threshold analysis results saved to ../models/threshold_analysis.csv")

# Save optimal threshold
optimal_threshold_info = {
    'optimal_threshold': float(optimal_threshold),
    'metrics_at_optimal_threshold': {
        'accuracy': float(threshold_df.loc[optimal_threshold_idx, 'accuracy']),
        'precision': float(threshold_df.loc[optimal_threshold_idx, 'precision']),
        'recall': float(threshold_df.loc[optimal_threshold_idx, 'recall']),
        'f1': float(threshold_df.loc[optimal_threshold_idx, 'f1']),
        'net_benefit': float(optimal_net_benefit)
    }
}

with open('../models/optimal_threshold.json', 'w') as f:
    json.dump(optimal_threshold_info, f)
print("Optimal threshold information saved to ../models/optimal_threshold.json")

## 10. Conclusion

In [ ]:
# Print final summary
print("\n=== Customer Churn Prediction Model Development Summary ===")
print(f"Best model: {best_model_name}")
print(f"ROC-AUC: {best_model_auc:.4f}")
print(f"Optimal threshold: {optimal_threshold:.2f}")
print(f"Net benefit at optimal threshold: ${optimal_net_benefit:,.0f}")
print("\nNext steps:")
print("1. Deploy the model in production")
print("2. Monitor model performance")
print("3. Update the model periodically with new data")
print("4. Implement targeted retention strategies based on model predictions")